In [5]:
# 安装pdfminer
!pip install pdfminer.six

In [ ]:
import os
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextBox, LTTextLine, LTChar, LTImage


# 读取 PDF 文件
file_directory = "not_upload\\关于开展公众市场三季度营销活动的通知.pdf"

print(os.getcwd())
# 获取当前目录
current_directory = os.getcwd()
# 获取当前目录的上上层目录
grandparent_directory = os.path.abspath(os.path.join(current_directory, "..\\.."))

file_path = os.path.join(grandparent_directory, file_directory)
print(file_path)




# 定义一个函数来提取并排序文本
def extract_and_sort_text(file_path):
    all_text = []

    # 提取每一页的布局信息
    for page_layout in extract_pages(file_path):
        # 按位置排序布局元素
        print(page_layout)
        sorted_elements = sorted(page_layout, key=lambda x: (x.y1, x.x0), reverse=True)
        print(sorted_elements)

        # 合并文本
        current_text = ""
        for element in sorted_elements:
            if isinstance(element, LTTextBox) or isinstance(element, LTTextLine):
                current_text += element.get_text()
            elif isinstance(element, LTImage):
                print("Image at position:", element.bbox)

        all_text.append(current_text.strip())

    return all_text


#提取并排序文本
all_text = extract_and_sort_text(file_path)

# 打印每一页的文本
for i, text in enumerate(all_text):
    print(f"Page {i + 1}:")
    print(text)
    print("\n")


### pdf页面读取时，保证读取顺序
```python
def extract_and_sort_text(file_path):
    all_text = []

    # 提取每一页的布局信息
    for page_layout in extract_pages(file_path):
        # 按位置排序布局元素
        print(page_layout)
        sorted_elements = sorted(page_layout, key=lambda x: (x.y1, x.x0), reverse=True)

        # 合并文本
        current_text = ""
        for element in sorted_elements:
            if isinstance(element, LTTextBox) or isinstance(element, LTTextLine):
                current_text += element.get_text()
            elif isinstance(element, LTImage):
                print("Image at position:", element.bbox)

        all_text.append(current_text.strip())

    return all_text
```

`sorted_elements = sorted(page_layout, key=lambda x: (x.y1, x.x0), reverse=True)` 

这段代码用来保证读取的pdf文档内容按顺序呈现，通过这种方式排列布局元素的原因有以下几点：
1. 在 pdfminer 中，元素坐标系统如下：
    - 原点 (0, 0)：位于页面的左下角。
    - x 轴：从左向右增加，一直到X轴边界。
    - y 轴：从下向上增加，一直到Y轴边界。
2. 排序目标
    - 首先按照 y1大小顺序， 将元素在Y轴上从上到下排序。reverse=True，即在Y轴方向上从PDF的上边界往Y轴的原点方向排列。
    - 当不同元素的y1相同时，则按x0从小到大顺序排列，即在X轴方向从PDF的X轴初始方向往右边界排列。
    - 经过以上在X，Y轴方向的排序步骤之后，sorted_elements 列表中存储的元素顺序符合人类查看PDF的习惯，从上到下，从左到右。
